In [2]:
from Tkinter import *
import tkFileDialog
import sys
import ConfigParser
import os
import logging
import re
from datetime import datetime

In [3]:
# This looks great!
# http://stackoverflow.com/questions/36604900/redirect-stdout-to-tkinter-text-widget

# also this:
# https://www.blog.pythonlibrary.org/2014/07/14/tkinter-redirecting-stdout-stderr/


# http://stackoverflow.com/questions/14883648/redirect-output-from-python-logger-to-tkinter-widget

In [4]:
# http://stackoverflow.com/questions/36604900/redirect-stdout-to-tkinter-text-widget
class IORedirector(object):
    '''A general class for redirecting I/O to this Text widget.'''
    def __init__(self,text_area):
        self.text_area = text_area
        
#     def flush(self):
#         pass

class StdoutRedirector(IORedirector):
    '''A class for redirecting stdout to this Text widget.'''

    def write(self,str):
        self.text_area.insert("end", str)
        self.text_area.see(END)

    def flush(self):
        pass

In [82]:
class myApp(object):
    def __init__(self, master):
        self.master = master
        master.title('Student Portfolio Populator')

        # get the configuration
        # Path to configuration files
        self.configPath = os.path.expanduser('~/.config/populate/')
        self.configFile = os.path.join(self.configPath, 'config.ini')
        self.getConfig()
        
        self.outputFile = datetime.now().strftime('Student_Data_Output_%Y%m%d-%H.%M.csv')
        
        self.msgReqURL = 'Paste Google Drive URL for Student Portfolio below:'
        self.msgOKURL = 'Student Portfolio URL accepted:'
        self.msgBadURL = 'Invalid URL format, please try again'
        self.message = self.msgReqURL
        
        self.studentFile = None
        self.studentData = None
        
        # launch the GUI interface
        self.configLogging()
        self.guiInit()
        
        
# consider moving this into its own def   
    def configLogging(self):
        # init the log; this removes any old log handlers (this is particularly useful when testing in an IDE)
        log = logging.getLogger()
        logging.getLogger("googleapiclient").setLevel(logging.WARNING)

        # useful for removing old log handlers when developing from an IDE such as Jupyter
        if len(log.handlers) > 0:
            for each in range(0, len(log.handlers)):
                log.removeHandler(log.handlers[0])


        # set the log format
        logFormatter = logging.Formatter('[%(levelname)8s %(asctime)s] %(message)s', '%Y-%m-%d %H:%M')
        #consoleFormatter = logging.Formatter('[%(levelname)-8s] %(message)s')
        consoleFormatter = logging.Formatter('[%(levelno)-3s] %(message)s')
        # set root logger
        self.rootLogger = logging.getLogger()       

        # set the logging level for the api discovery service to "ERROR"
        logging.getLogger('discovery').setLevel(logging.ERROR)


        # add a conshole handle to the root logger
        consoleHandler = logging.StreamHandler(sys.stdout)
    #     consoleHandler.setFormatter(logFormatter)
        consoleHandler.setFormatter(consoleFormatter)
        self.rootLogger.addHandler(consoleHandler) 

        # Set default logging level
        self.rootLogger.setLevel(logging.DEBUG)
        logging.error('just kidding, no error')
    #     rootLogger.setLevel(logging.WARNING)
    #    rootLogger.setLevel(logging.INFO)

    def guiInit(self):
#         initiate the output window 
        self.redirector('Student Import Messages:\n')

        # set window size
        self.master.geometry('800x400+30+30')
#         self.label = Label(self.master, text = 'Paste URL for Student Portfolio')

        self.labelURL_text = StringVar()
        self.labelURL_text.set(self.message)
        self.labelURL = Label(self.master, textvariable = self.labelURL_text, font = "TkDefaultFont 0 bold")

        self.labelStuFileRow_text = StringVar()
        self.labelStuFileRow_text.set('PowerSchool student information to import (student_export.text): ')
        self.labelStuFileRow = Label(self.master, textvariable = self.labelStuFileRow_text, 
                                     font = "TkDefaultFont 0 bold")

        self.labelStuFile_text = StringVar()
        self.labelStuFile_text.set('None selected')
        self.labelStuFile = Label(self.master, textvariable = self.labelStuFile_text)

        self.labelOutpathRow_text = StringVar()
        self.labelOutpathRow_text.set('Location to output portfolio URLs:')
        self.labelOutpathRow = Label(self.master, textvariable = self.labelOutpathRow_text,
                                 font = "TkDefaultFont 0 bold")
        
        self.labelOutFolder_text = StringVar()
        self.labelOutFolder_text.set(self.outputFolder)
        self.labelOutFolder = Label(self.master, textvariable = self.labelOutFolder_text)
        
        self.submit_button = Button(self.master, text = 'BEGIN FOLDER CREATION', command = self.submit)
        self.quit_button = Button(self.master, text = 'quit', command = self.quit)
        self.buttonStFile = Button(self.master, text = 'set student data file', command = self.studentFilePicker)
        self.buttonOutFolder = Button(self.master, text = 'set output folder', command = self.outputFolderPicker)
        
        
        self.entryURL = Entry(self.master, width = 60, validate='key') #, validatecommand=(vcmd, '%P'))
        self.entryURL.insert(END, self.gdBaseFolder)

#         # add menubar to master window
#         menubar = Menu(self.master)
#         self.master.config(menu=menubar)
        
#         filemenu = Menu(menubar)
#         menubar.add_cascade(label='Log Level', menu=filemenu)

#         filemenu.add_command(label='Errors', command = self.setLogLevel('ERROR'))
#         filemenu.add_command(label='Warnings', command = self.setLogLevel('WARN'))
#         filemenu.add_command(label='Info', command = self.setLogLevel('INFO'))
#         filemenu.add_command(label='Debug', command = self.setLogLevel('DEBUG'))

        
        ############ organize the grid
        self.labelURL.grid(row = 0, columnspan = 2, sticky=W)        
       
        self.entryURL.grid(row = 2, column = 0, columnspan = 2, sticky = W)
        
        self.labelStuFileRow.grid(row = 3, column =0, sticky = W)
        self.labelStuFile.grid(row = 3, column = 1, sticky = W)        
        self.buttonStFile.grid(row = 4, column = 0, sticky = W+E)
        
        self.labelOutpathRow.grid(row = 5, column = 0, sticky = W)
        self.labelOutFolder.grid(row = 5, column = 1, sticky = W)
        self.buttonOutFolder.grid(row = 6, column =0, sticky = W+E)
        
        self.submit_button.grid(row = 7, column = 0, sticky = W+E)
        
        self.quit_button.grid(row = 8, column = 0, sticky = W)  
        
    def setLogLevel(self, level): 
        self.loglevel = level
        self.rootLogger.setLevel(getattr(logging, level))
    
    def getConfig(self):
        '''
        Reads configuration file and sets the following attributes:
            self.gdBaseFolder
            self.outputFolder
            self.loglevel'''
        
        self.parser = ConfigParser.SafeConfigParser()
        
        # required options in the 'Main' section
        self.mainSection = 'Main'
        # required key: [method for getting, default value]
        prefs = {'gdBaseFolder': [self.parser.get, 'https://drive.google.com/SAMPLE/URL/abc123zyx987'],
                 'outputFolder': [self.parser.get, os.path.expanduser('~/Desktop/')],
                 'loglevel': [self.parser.get, 'INFO'],
                 'googleCreds': [self.parser.get, os.path.join(self.configPath, 'GDrive-python_creds.json')]}
        
        # make sure a configuration path exists
        if len(self.parser.read(self.configFile)) <= 0:
            logging.warn('no configuration files found at: {}'.format(self.configFile))
            logging.debug('creating configuration files')
            try:
                os.makedirs(os.path.expanduser(self.configPath))
            except OSError as e:
                if e.errno != 17:
                    logging.critical(e)
                    sys.exit(1)
        
        # make sure there is a main section
        if not self.parser.has_section(self.mainSection):
            self.parser.add_section(self.mainSection)
        
        self.preferences = {}
        
        # read search for the expected preferences in the configuration file
        # note which are missing and set to the default values above
        for key in prefs:
            try:
                self.preferences[key] = prefs[key][0](self.mainSection, key)
            except (ConfigParser.NoSectionError, ConfigParser.NoOptionError):
                self.parser.set(self.mainSection, key, prefs[key][1])
                self.preferences[key] = prefs[key][1]
        
        self.gdBaseFolder = self.preferences['gdBaseFolder']
        self.outputFolder = self.preferences['outputFolder']
        self.loglevel = self.preferences['loglevel']
        self.googleCreds = self.preferences['googleCreds']
        
        
    def outputFolderPicker(self):
        try:
            folder = tkFileDialog.askdirectory(title = 'Choose output folder')

        except (IOError, OSError) as e:
            print e
            return(False)
        
 
        if len(folder) > 0:
            self.outputFolder = folder
            self.labelOutFolder_text.set(self.outputFolder)        
    
    def studentFilePicker(self):
        try:
            self.studentFile = tkFileDialog.askopenfile(mode='r', title='Choose exported student data')
        except (IOError, OSError) as e:
            print e
            return(False)
        
        
        try:
            self.studentData = self.studentFile.read()
            self.labelStuFile_text.set(self.studentFile.name)
            print self.studentData
        except AttributeError:
            pass
        
        return(True)
    
#     def spawnNew(self):
#         win = Toplevel()
#         win.title('stuff')
#         message = Message(win, text ="some more stuff")
#         message.pack()

    def redirector(self, inputStr=""):
        wX = 800
        wY = 600
        geometry = '{}x{}+40+350'.format(wX, wY)
        import sys
        root = Toplevel()
#         root.lift()
        root.geometry(geometry)
        T = Text(root, font = 'Courier 14', foreground='white', width = wX, height = wY)
        T.config(background='black')
        sys.stdout = StdoutRedirector(T)
        T.pack()
        T.insert(END, inputStr)
        
    def submit(self):
        logging.error('starting submission procedure')
        # check for a valid URL format
        urlRegex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
        
        # spawn a text window for output
        if re.match(urlRegex, self.entryURL.get()): 
            self.portfolioURL = self.entryURL.get()
            
            self.message = self.msgOKURL
        else: 
            self.portfolioURL = ''
            self.message = self.msgBadURL        
        
        self.labelURL_text.set(self.message)
        
        self.parser.set(self.mainSection, 'gdBaseFolder', self.portfolioURL)
        self.parser.set(self.mainSection, 'outputFolder', self.outputFolder)        
        
        # write configuration file
        logging.debug('writing configuration to file: {}'.format(self.configFile))
        try:
            self.parser.write(open(self.configFile, 'w'))
        except Exception as e:
            logging.error('Error writing configuration file: {}'.format(e))
        
        
        # kick off actual task 
        # gDrivePopulate(base folder url, path to credentials etc.)
            
        
    def greet(self):
        print('hello world')
        
    def quit(self):
        self.master.destroy()

root = Tk()
my_gui = myApp(root)
root.mainloop()
sys.stdout.flush()

Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/logging/__init__.py", line 882, in emit
    stream.write(fs % msg)
  File "<ipython-input-4-d7b3ca8a2fa6>", line 14, in write
    self.text_area.insert("end", str)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/lib-tk/Tkinter.py", line 3110, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
TclError: invalid command name ".4924520856.4425288592"
Logged from file <ipython-input-82-d40003cf55ce>, line 58


In [7]:
my_gui.getConfig()


In [ ]:
my_gui.parser.read(my_gui.configFile)

In [6]:
my_gui.parser.set('Main', 'basefolder', 'https://drive.google.com/drive/folders/0B9WTleJ1MzaYcmdmTWNNNF9pa1E')
#my_gui.configfile = os.path.expanduser('~/.prefs/populate/')
#my_gui.parser.write(open(my_gui.configfile, 'w'))

In [ ]:
prefsPath = '~/.prefs/populate/'
try:
    os.makedirs(os.path.expanduser(prefsPath))
except OSError as e:
    if e.errno != 17:
        print e
    

In [ ]:
import errno
errno.errorcode.keys()
errno.errorcode

In [ ]:
root.mainloop()
sys.stdout.flush()

In [ ]:
my_gui.labelURL.cget()

In [ ]:
sys.stdout.flush()
printstuff()

In [ ]:
import re

In [ ]:
urlRegex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

In [67]:
string = 'ftp://google.gu'
if re.match(regex, string):
    print 'ok!'
else:
    print 'crap'

NameError: name 'regex' is not defined

In [74]:
logger =logging.getLogger()
logger.setLevel(getattr(logging, 'foo'))


AttributeError: 'module' object has no attribute 'foo'